# Figures for field data FWI
and matching filters used in the thesis

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
from scipy import stats
import math
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib import patches
from matplotlib.ticker import (FormatStrFormatter,MultipleLocator, AutoMinorLocator)

import seppy
import os

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotvel(d, vmin=-3, vmax=3, xmin=0, xmax=2.05, zmin=1.92, zmax=1.96, cmap="seismic", clabel = "%", grid=True, hrz=np.array([]), srxz=np.array([]), plot_rec=True, savefig=None):
    """Plot Vp and Vs models"""

    fig, ax = plt.subplots(2,1,figsize=(10, 4),dpi=300)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=0.25)

    nx = d.shape[1]
    dx = 0.002

    im0=ax[0].imshow(np.transpose(d[0,:,:]),interpolation='bilinear',aspect="auto",extent=[xmin,xmax,zmax,zmin],cmap="seismic",vmin=vmin,vmax=vmax)
    ax[0].set(ylabel='Depth (km)')
    ax[0].set_xticklabels([])
    ax[0].set_title(r"a) $V_{\mathrm{P}0}$ perturbation",loc='left')
    ax[0].tick_params(axis='both', which='major', labelsize=6)
    # ellipse = patches.Ellipse(xy=(0.61, 1.94), width=0.5, height=0.02, edgecolor='k', fc='None', lw=1, ls='--')
    # ellipse = patches.Ellipse(xy=(0.61, 1.944), width=0.5, height=0.01, edgecolor='k', fc='None', lw=1, ls='--')
    # ax[0].add_patch(ellipse)
    # ax[0].annotate("", xy=[0.93,1.948],xytext=[0.9,1.958],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
    if grid==True:
        ax[0].grid(color='k', linestyle='-', linewidth=0.2)
    if hrz.size>0:
        ax[0].plot(np.arange(xmin,nx*dx,dx),hrz,c='k',linewidth=0.5,linestyle='--')
    if srxz.size>0:
        if plot_rec==True:
            ax[0].scatter(srxz[:,:,3], srxz[:,:,4], s=0.5, c='y', marker=',',linewidths=0.5)
            plt.ylim([zmax,zmin])
        ax[0].scatter(srxz[:,0,1], srxz[:,0,2], s=20, c='m', marker='x',linewidths=1)

    im1=ax[1].imshow(np.transpose(d[1,:,:]),interpolation='bilinear',aspect="auto",extent=[xmin,xmax,zmax,zmin],cmap="seismic",vmin=vmin,vmax=vmax)
    ax[1].set(xlabel='X (km)',ylabel='Depth (km)')
    ax[1].set(xlabel='X (km)',ylabel='Depth (km)')
    ax[1].set_title(r"b) $V_{\mathrm{S}0}$ perturbation",loc='left')
    ax[1].tick_params(axis='both', which='major', labelsize=6)
    # ellipse = patches.Ellipse(xy=(0.61, 1.945), width=0.5, height=0.01, edgecolor='k', fc='None', lw=1, ls='--')
    # ax[1].add_patch(ellipse)
    # ax[1].annotate("", xy=[0.93,1.948],xytext=[0.9,1.958],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
    if grid==True:
        ax[1].grid(color='k', linestyle='-', linewidth=0.2)
    if hrz.size>0:
        ax[1].plot(np.arange(xmin,nx*dx,dx),hrz,c='k',linewidth=0.5,linestyle='--')
    if srxz.size>0:
        if plot_rec==True:
            ax[1].scatter(srxz[:,:,3], srxz[:,:,4], s=0.5, c='y', marker=',',linewidths=0.5)
            plt.ylim([zmax,zmin])
        ax[1].scatter(srxz[:,0,1], srxz[:,0,2], s=20, c='m', marker='x',linewidths=1)

    cbar=fig.colorbar(im0, ax=ax, orientation='vertical',shrink=0.7, format="%.0f",pad=0.01)
    cbar.ax.tick_params(labelsize=6)
    cbar.set_label(label="%",size=6)

    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()
        
def plotseis_interleave(d1, d2, inter=50, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=-650, xmax=-150, grid=True, colorbar=False, savefig=None):
    """Plot 2D seismic image interleaving traces from 2 datasets"""
 
    assert(d1.shape == d2.shape)
    
    vmin=np.min(d1) / p
    vmax=p*np.max(d1)
    
    if p<0:
        vmax=-p*np.amax(d1)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax

    if d1.ndim==2:
        ns = 1
        nx = d1.shape[0]
        nt = d1.shape[1]
        d1 = np.reshape(d1, (ns,nx,nt))
        d2 = np.reshape(d2, (ns,nx,nt))
    else:
        ns=d1.shape[0]
        nx=d1.shape[1]
        nt=d1.shape[2]

    d = np.copy(d1)
    flag=np.zeros((nx))
    itr1=0
    itr2=inter
    while (itr2<=nx):
        d[:,itr1:itr2,:] = d2[:,itr1:itr2,:]
        flag[itr1:itr2]=0.01
        itr1 += 2*inter
        itr2 += 2*inter
    if itr1<nx:
        d[:,itr1:,:] = d2[:,itr1:,:]
                
    fig=plt.figure(figsize=(8, 6),dpi=300)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.08, hspace=0.)
    for s in range(ns):
        plt.subplot(1,ns,s+1)
        plt.imshow(np.transpose(d[s]),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)   
        plt.plot(np.arange(xmin,xmax+1),flag,linewidth=1,color='r')
        plt.xlabel('Offset (m)')
        plt.gca().tick_params(axis='both', which='major', labelsize=6)
        plt.gca().xaxis.set_minor_locator(MultipleLocator(inter))
        if s==0:
            plt.ylabel('Time (s)')
            plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        else:
            plt.gca().set_yticklabels([])
        if grid==True:
            plt.grid(color='w', linestyle='-', linewidth=0.2, which='both')
    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()

def plotfunc(x, y, ymin=0, ymax=1, yscale="linear", ylabel="Value", labels=[], legend=True, savefig=None):
    """Plot misfit functions"""

    if len(labels)>0:
        assert (len(labels) == len(y))
    else:
        for i in range(len(y)):
            label = "Curve "+str(i+1)
            labels.append(label)
                
    fig=plt.figure(figsize=(4.33, 2),dpi=300)

    for i in range(len(y)):
        plt.plot(x[i],y[i],label=labels[i],Linewidth=1)
    if legend==True:
        plt.legend()
    plt.xlabel("Iteration number")
    plt.ylabel(ylabel)
    if ymin!=ymax:
        plt.ylim([ymin,ymax])
    plt.yscale(yscale)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)

    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()

def getXYZ(md, traj,md0=-1):
    """Compute x,y,z coordinates for a given measured depth"""

    n = traj.shape[1]
    
    if md0<0:
        l=traj[2,0]
    else:
        l=md0
    i=1
    while ((l<md) and (i<n)):
        l += math.sqrt((traj[2,i]-traj[2,i-1])**2+(traj[1,i]-traj[1,i-1])**2+(traj[0,i]-traj[0,i-1])**2)
        i += 1
        
    if (i==n):
        print("Warning: the end of the trajectory has been reached")
        x=traj[0,n-1]
        y=traj[1,n-1]
        z=traj[2,n-1]
    else:
        imin = i - 2
        imax=imin+1
        if (imin<0):
            imin=0
            imax=0
        lmin=l-math.sqrt((traj[2,imax]-traj[2,imax-1])**2+(traj[1,imax]-traj[1,imax-1])**2+(traj[0,imax]-traj[0,imax-1])**2)
        wmax=(md-lmin)/(l-lmin)
        wmin=1-wmax
        x=wmin*traj[0,imin]+wmax*traj[0,imax]
        y=wmin*traj[1,imin]+wmax*traj[1,imax]
        z=wmin*traj[2,imin]+wmax*traj[2,imax]
    
    return x,y,z

def ChannelToDepth(Channel):
    """Compute DAS channel measured depth from channel number (between 1 and 3776)
    The conversion function is provided by Chevron"""
    
    FibreLengthMultiplier = 1.02095238387
    ZeroOffset_m          =-2.41333217886
    SpatialResolution_m   = 1.0
    DasFibreStretch       = 1.0056

    Depth_in_meters       = ( (float(Channel) 
                             * SpatialResolution_m 
                             * FibreLengthMultiplier )
                             + ZeroOffset_m) / DasFibreStretch

    return Depth_in_meters

def rotationY(angle):
    """Build rotation matrix around y-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,0,-s],[0,1,0],[s,0,c]))

    return R

def rotationZ(angle):
    """Build rotation matrix around z-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,-s,0],[s,c,0],[0,0,1]))

    return R

def rotateYZ(xyz,theta,phi):
    """Rotate coordinates around y- and z-axes ; angles are in degrees"""

    Ry = rotationY(theta)
    Rz = rotationZ(phi)
    v = np.matmul(Rz,xyz)
    v = np.matmul(Ry,v)

    return v

def get_profile(mod, rx, rz, Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0):
    """Extract values from model using bilinear interpolation"""

    n=rx.shape[0]
    modp=np.zeros((5,n))
    for i in range(n):
        xval=max(rx[i],Xo)
        zval=max(rz[i],Zo)
        ix = math.floor((xval-Xo)/Xd)
        iz = math.floor((zval-Zo)/Zd)
        ix=min(ix,Xn-2)
        iz=min(iz,Zn-2)
        wx = (xval - ix*Xd-Xo)/Xd
        wz = (zval - iz*Zd-Zo)/Zd
        if nzsum<=0:
            modp[:,i] = (1-wx)*(1-wz)*mod[:,ix,iz] + (1-wx)*wz*mod[:,ix,iz+1] + wx*(1-wz)*mod[:,ix+1,iz] + wx*wz*mod[:,ix+1,iz+1]
        else:
            iz = int(round((zval-Zo)/Zd))
            modp[:,i] = (1-wx)*np.average(mod[:,ix,iz-nzsum+1:iz+nzsum],axis=1) + wx*np.average(mod[:,ix+1,iz-nzsum+1:iz+nzsum],axis=1)
        
    return modp

def get_profile_hrz(mod, hrz, Xn, Xo, Xd, Zn, Zo, Zd):
    """Extract values from model along horizon"""

    n=mod.shape[1]
    modp=np.zeros((5,n))
    for i in range(n):
        iz = round((hrz[i]-Zo)/Zd)
        modp[:,i] = mod[:,i,iz]
        
    return modp

### 2D models

In [ ]:
# read attributes, mask, models, and horizon

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.HH")
pattr = data.reshape(data_axes.n,order='F').T

data_axes, data = sep.read_file(datapath+"ch5st_data_group2.HH")
pattr_st = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_hrz.H")
ch5_hrz = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_mask.H")
mask = data.reshape(axes.n,order='F').T
mask = mask[0,:,40:81]

axes, data = sep.read_file(datapath+"ch5_model.H")
ch5_model = data.reshape(axes.n,order='F').T
ch5_model = ch5_model[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_model_2d.H")
ch5un_model_2d = data.reshape(axes.n,order='F').T
ch5un_model_2d = ch5un_model_2d[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi1d.H")
ch5un_fwi1d = data.reshape(axes.n,order='F').T
ch5un_fwi1d = ch5un_fwi1d[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_1.H")
ch5un_fwi2d_1 = data.reshape(axes.n,order='F').T
ch5un_fwi2d_1 = ch5un_fwi2d_1[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_2a.H")
ch5un_fwi2d_2a = data.reshape(axes.n,order='F').T
ch5un_fwi2d_2a = ch5un_fwi2d_2a[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_2b.H")
ch5un_fwi2d_2b = data.reshape(axes.n,order='F').T
ch5un_fwi2d_2b = ch5un_fwi2d_2b[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_2c.H")
ch5un_fwi2d_2c = data.reshape(axes.n,order='F').T
ch5un_fwi2d_2c = ch5un_fwi2d_2c[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_3a.H")
ch5un_fwi2d_3a = data.reshape(axes.n,order='F').T
ch5un_fwi2d_3a = ch5un_fwi2d_3a[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5un_fwi2d_4a.H")
ch5un_fwi2d_4a = data.reshape(axes.n,order='F').T
ch5un_fwi2d_4a = ch5un_fwi2d_4a[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5st_model0_2d.H")
ch5st_model0_2d = data.reshape(axes.n,order='F').T
ch5st_model0_2d = ch5st_model0_2d[:,:,40:81]

axes, data = sep.read_file(datapath+"ch5st_fwi2d_0e_group2.H.ext")
ch5st_fwi2d_0e_group2_ext = data.reshape(axes.n,order='F').T
ch5st_fwi2d_0e_group2_ext = ch5st_fwi2d_0e_group2_ext[:,:,:,40:81]

axes, data = sep.read_file(datapath+"ch5st_fwi2d_2.H")
ch5st_fwi2d_2 = data.reshape(axes.n,order='F').T
ch5st_fwi2d_2 = ch5st_fwi2d_2[:,:,40:81]

ntr=data_axes.n[1]
ns=data_axes.n[2]
Zn=ch5un_model_2d.shape[-1]
Xn=axes.n[1]
Zo=1.92
Xo=axes.o[1]
Zd=axes.d[0]
Xd=axes.d[1]

zmin=Zo
zmax=Zo+(Zn-1)*Zd

srxz=np.zeros((ns,ntr,6))
srxz[:,:,0]=pattr[:,:,0]
srxz[:,:,1]=pattr[:,:,19]*1e-5
srxz[:,:,2]=pattr[:,:,21]*1e-5
srxz[:,:,3]=pattr[:,:,16]*1e-5
srxz[:,:,4]=pattr[:,:,18]*1e-5
srxz[:,:,5]=pattr[:,:,24]

srxz_st=np.zeros((ns,ntr,6))
srxz_st[:,:,0]=pattr_st[:,:,0]
srxz_st[:,:,1]=pattr_st[:,:,19]*1e-5
srxz_st[:,:,2]=pattr_st[:,:,21]*1e-5
srxz_st[:,:,3]=pattr_st[:,:,16]*1e-5
srxz_st[:,:,4]=pattr_st[:,:,18]*1e-5
srxz_st[:,:,5]=pattr_st[:,:,24]

In [ ]:
# starting model with unstimulated geometry

fig=plt.figure(figsize=(10, 4),dpi=300)
plt.imshow(np.transpose(ch5_model[0,:,:]),interpolation='bilinear',aspect="auto",extent=[0,2.05,zmax,zmin],cmap="jet",vmin=2.5,vmax=6.0)
plt.ylim([zmax,zmin])
plt.xlabel("X (km)")
plt.ylabel("Depth (km)")
plt.tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
cbar = plt.colorbar(shrink=0.7, format="%.1f",pad=0.01)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="(km/s)",size=6)
plt.scatter(srxz[:,:,3], srxz[:,:,4], s=0.5, c='y', marker=',')
plt.scatter(srxz[:,0,1], srxz[:,0,2], s=20, c='m', marker='x')
plt.plot(np.arange(Xo,Xn*Xd,Xd),ch5_hrz,c='k',linewidth=1,linestyle='--')

plt.savefig(figpath+'ch5un_model_vp.png',bbox_inches='tight',format='png')

In [ ]:
# starting model with stimulated geometry

fig=plt.figure(figsize=(10, 4),dpi=300)
plt.imshow(np.transpose(ch5_model[0,:,:]),interpolation='bilinear',aspect="auto",extent=[0,2.05,zmax,zmin],cmap="jet",vmin=2.5,vmax=6.0)
plt.ylim([zmax,zmin])
plt.xlabel("X (km)")
plt.ylabel("Depth (km)")
plt.tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
cbar = plt.colorbar(shrink=0.7, format="%.1f",pad=0.01)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="(km/s)",size=6)
plt.scatter(srxz_st[:,:,3], srxz_st[:,:,4], s=0.5, c='y', marker=',')
plt.scatter(srxz_st[:,0,1], srxz_st[:,0,2], s=20, c='m', marker='x')
plt.plot(np.arange(Xo,Xn*Xd,Xd),ch5_hrz,c='k',linewidth=1,linestyle='--')

plt.savefig(figpath+'ch5st_model_vp.png',bbox_inches='tight',format='png')

In [ ]:
# starting model with stimulated geometry, single shot
sp=9

fig=plt.figure(figsize=(10, 4),dpi=300)
plt.imshow(np.transpose(ch5_model[0,350:901,:]),interpolation='bilinear',aspect="auto",extent=[0.75,1.8,zmax,zmin],cmap="jet",vmin=2.5,vmax=6.0)
plt.ylim([zmax,zmin])
plt.xlabel("X (km)")
plt.ylabel("Depth (km)")
plt.tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
cbar = plt.colorbar(shrink=0.7, format="%.1f",pad=0.01)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="(km/s)",size=6)
plt.scatter(srxz_st[sp,:,3], srxz_st[sp,:,4], s=0.5, c='y', marker=',')
plt.scatter(srxz_st[sp,0,1], srxz_st[sp,0,2], s=80, c='m', marker='x')

plt.savefig(figpath+'ch5st_model_vp_sp.png',bbox_inches='tight',format='png')

In [ ]:
# reference FWI
d=100*(ch5un_fwi2d_2a[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=30
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi2d_2a")

In [ ]:
# reference FWI with limited number of iterations

d=100*(ch5un_fwi2d_2b[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=10
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi2d_2b")

In [ ]:
# extended FWI

d=100*(ch5un_fwi2d_3a[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=3
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi2d_3a")

In [ ]:
# extended FWI with B-splines

d=100*(ch5un_fwi2d_1[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=3
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi2d_1")

In [ ]:
# reference FWI with B-splines

d=100*(ch5un_fwi2d_2c[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=6
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig='ch5un_fwi2d_2c')

In [ ]:
# 1.5D FWI with B-splines

d=100*(ch5un_fwi1d[0:2,:,:] - ch5_model[0:2,:,:])/ch5_model[0:2,:,:]
vmax=3
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi1d")

In [ ]:
# extended FWI with B-splines at HF

d=100*(ch5un_fwi2d_4a[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=5
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5un_fwi2d_4a")

In [ ]:
# example of garbage collector model
sp=9
d=100*(ch5st_fwi2d_0e_group2_ext[sp,0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=30
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, srxz=srxz_st[sp:sp+1,:51,:], savefig="ch5st_fwi2d_0e_sp")

In [ ]:
# extended FWI with B-splines at HF, stimulated side

d=100*(ch5st_fwi2d_2[0:2,:,:] - ch5un_model_2d[0:2,:,:])/ch5un_model_2d[0:2,:,:]
vmax=5
plotvel(d, vmin=-vmax, vmax=vmax, grid=True, hrz=ch5_hrz, savefig="ch5st_fwi2d_2")

In [ ]:
# histograms

istart=250
iend=875
bins=np.linspace(-70,80,16)-5

fig=plt.figure(figsize=(8, 6),dpi=300)

plt.subplot(2,1,1)

vals=(1000*mask[istart:iend,:]*(ch5un_fwi2d_4a[0,istart:iend,:]-ch5un_model_2d[0,istart:iend,:])).flatten()
vals_st=(1000*mask[istart:iend,:]*(ch5st_fwi2d_2[0,istart:iend,:]-ch5un_model_2d[0,istart:iend,:])).flatten()
vals = vals[np.abs(vals)>5]
vals_st = vals_st[np.abs(vals_st)>5]

plt.hist(vals, bins=bins, density=True, alpha=1, label='Pre-stimulation', color='b')
plt.hist(vals_st, bins=bins, density=True, alpha=0.5, label='Post-stimulation', color='y')
plt.legend()
plt.gca().set_xticklabels([])
plt.ylabel(r'Normalized count')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $V_{\mathrm{P}0}$ perturbation', loc='left')

distance = stats.wasserstein_distance(vals_st,vals)
median_distance = np.median(vals_st) - np.median(vals)
print("Wasserstein distance between the two distributions is %.3f" %distance)
print("Median distance between the two distributions is %.0f m/s" %median_distance)

plt.subplot(2,1,2)

vals=(1000*mask[istart:iend,:]*(ch5un_fwi2d_4a[1,istart:iend,:]-ch5un_model_2d[1,istart:iend,:])).flatten()
vals_st=(1000*mask[istart:iend,:]*(ch5st_fwi2d_2[1,istart:iend,:]-ch5un_model_2d[1,istart:iend,:])).flatten()
vals = vals[np.abs(vals)>5]
vals_st = vals_st[np.abs(vals_st)>5]

plt.hist(vals, bins=bins, density=True, alpha=1, label='Pre-stimulation', color='b')
plt.hist(vals_st, bins=bins, density=True, alpha=0.5, label='Post-stimulation', color='y')
plt.legend()
plt.xlabel(r'Perturbation (m/s)')
plt.ylabel(r'Normalized count')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $V_{\mathrm{S}0}$ perturbation', loc='left')

distance = stats.wasserstein_distance(vals_st,vals)
median_distance = np.median(vals_st) - np.median(vals)
print("Wasserstein distance between the two distributions is %.3f" %distance)
print("Median distance between the two distributions is %.0f m/s" %median_distance)

plt.savefig(figpath+'ch5st_histograms.png',bbox_inches='tight',format='png')

### Well profiles

In [ ]:
# Read the deviated well (monitor) trajectory
# The file contains 4 columns: Measured Depth (ft), Inclination (degrees), NS (ft), EW (ft)

inputfile="../../../input_data/auxiliary_data/MonitorWell_WellSurvey.csv"

df=pd.read_csv(inputfile, sep=',',header=0)
traj=df.values[:,0:4]

# convert the MD to TVD and convert (ft) to (m)
n=traj.shape[0]
z = np.zeros(n)

x = traj[:,3]*0.3048
y = traj[:,2]*0.3048

z[0] = 0
for i in range(1, n):
    z[i] = z[i-1] + (traj[i,0]-traj[i-1,0])*math.cos(math.radians(traj[i,1]))

z = z*0.3048

# save the trajectory
trajectory = np.array([x,y,z])

# conversion table between MD and TVD
tvd = np.zeros((n, 2))
tvd[:,0] = 0.3048*traj[:,0]
tvd[:,1] = z


#Read the deviated well log
#The file contains 7 columns: Measured Depth (ft), Borehole deviation (degrees), P slowness (us/ft), 
#S-fast slowness (us/ft), S-slow slowness (us/ft), Bulk density modeled (g/cc), Normalized Gamma ray (unitless)

inputfile="../../../input_data/auxiliary_data/MonitorWell_WellLog.csv"

df=pd.read_csv(inputfile, sep=',',header=2)
wlog=df.values[:,0:7]
wlog=np.transpose(wlog)

# convert measured depth to (m)
wlog[0,:] *= 0.3048

# convert slownesses to velocity (km/s)
wlog[2,:] = 0.3048*1e+03/wlog[2,:] # Vp
wlog[3,:] = 0.3048*1e+03/wlog[3,:] # Vs fast
wlog[4,:] = 0.3048*1e+03/wlog[4,:] # Vs slow

# rotate the coordinates
theta = -0.648114658
phi = 54.4

n = wlog.shape[1]
w_rotxyz=np.zeros((n,3))
for i in range(w_rotxyz.shape[0]):
    w_rotxyz[i,0],w_rotxyz[i,1],w_rotxyz[i,2] = getXYZ(wlog[0,i], trajectory)
    w_rotxyz[i,:] = rotateYZ(w_rotxyz[i,:],theta,phi)
 
# convert measured depth to km and coordinates to km
wlog[0,:] *= 0.001
w_rotxyz *= 0.001

In [ ]:
#Extract values from model using bilinear interpolation
ch5un_model_2d_p=get_profile(ch5un_model_2d,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)
ch5un_fwi2d_1_p=get_profile(ch5un_fwi2d_1,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)
ch5un_fwi2d_2a_p=get_profile(ch5un_fwi2d_2a,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)
ch5un_fwi2d_2b_p=get_profile(ch5un_fwi2d_2b,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)
ch5un_fwi2d_2c_p=get_profile(ch5un_fwi2d_2c,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)
ch5un_fwi2d_3a_p=get_profile(ch5un_fwi2d_3a,w_rotxyz[:,0],w_rotxyz[:,2],Xn, Xo, Xd, Zn, Zo, Zd, nzsum=0)

# smooth the well log
wlog_smth=np.copy(wlog)
kernel_size = 21
kernel = np.ones(kernel_size) / kernel_size
kernel_dirac = np.zeros(kernel_size)
kernel_dirac[0]=1
for i in range(2,7,1):
    wlog_smth[i,:] = np.convolve(wlog[i,:], kernel, mode='same')

In [ ]:
# Average profiles inside the reservoir
ch5un_model_2d_avg = np.sum(mask[:,:]*ch5un_model_2d[:,:,:],axis=-1)/np.sum(mask[0,:])
ch5un_fwi2d_1_avg = np.sum(mask[:,:]*ch5un_fwi2d_1[:,:,:],axis=-1)/np.sum(mask[0,:])
ch5un_fwi2d_2a_avg = np.sum(mask[:,:]*ch5un_fwi2d_2a[:,:,:],axis=-1)/np.sum(mask[0,:])
ch5un_fwi2d_3a_avg = np.sum(mask[:,:]*ch5un_fwi2d_3a[:,:,:],axis=-1)/np.sum(mask[0,:])

In [ ]:
# Average epsilon inside the reservoir
epsilon_avg = np.sum(mask[0,:]*ch5un_model_2d[4,0,:])/np.sum(mask[0,:])
factor=math.sqrt(1+2*epsilon_avg)
print("Epsilon average in the reservoir = %f" %epsilon_avg)

In [ ]:
Xarray0 = np.linspace(Xo,(Xn-1)*Xd+Xo,Xn)
Xarray = w_rotxyz[:,0]

p1=wlog
p2=ch5un_model_2d_p
p3=ch5un_fwi2d_2a_p
p4=ch5un_fwi2d_3a_p

p1_avg = ch5un_model_2d_avg
p2_avg = ch5un_fwi2d_3a_avg

fig=plt.figure(figsize=(10, 4),dpi=300)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.25, hspace=0.25)

plt.subplot(2,1,1)
plt.plot(Xarray,p1[2],       linewidth=0.5, c='b', label="Well log")
plt.plot(Xarray,factor*p2[0],linewidth=2.5, c='k', label="Initial")
plt.plot(Xarray,factor*p3[0],linewidth=0.3, c='g', label="Conventional FWI")
plt.plot(Xarray,factor*p4[0],linewidth=1.2, c='r', label="Extended FWI")
plt.legend(fontsize=6,loc='upper right')
plt.xlim([0,2.05])
plt.ylim([3,5])
plt.gca().set_xticklabels([])
plt.ylabel(r'Velocity (km/s)')
plt.title(r'a) $V_{\mathrm{P}90}$',loc='left')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='k', linestyle='-', linewidth=0.2)

plt.subplot(2,1,2)
plt.plot(Xarray,p1[4],linewidth=0.5, c='b', label="Well log")
plt.plot(Xarray,p2[1],linewidth=2.5, c='k', label="Initial")
plt.plot(Xarray,p3[1],linewidth=0.3, c='g', label="Conventional FWI")
plt.plot(Xarray,p4[1],linewidth=1.2, c='r', label="Extended FWI")
plt.legend(fontsize=6,loc='upper right')
plt.xlim([0,2.05])
plt.ylim([1.4,2.])
plt.xlabel(r'X (km)')
plt.ylabel(r'Velocity (km/s)')
plt.title(r'b) $V_{\mathrm{S}0}$',loc='left')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='k', linestyle='-', linewidth=0.2)
plt.annotate("", xy=[0.28,1.7],xytext=[0.32,1.8],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("", xy=[0.485,1.75],xytext=[0.53,1.85],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("", xy=[0.64,1.75],xytext=[0.61,1.87],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("", xy=[0.85,1.53],xytext=[0.8,1.45],color='w', arrowprops=dict(edgecolor='r',facecolor='white', width=1, headlength=5, headwidth=4))

plt.savefig(figpath+'ch5un_profiles.png',bbox_inches='tight',format='png')

### Misfit function

In [ ]:
# read misfit functions

axes, data = sep.read_file(datapath+"ch5un_fwi2d_2a.H.func")
ch5un_fwi2d_2a_func = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_2c.H.func")
ch5un_fwi2d_2c_func = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_1.H.func")
ch5un_fwi2d_1_func = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_1.H.func.d")
ch5un_fwi2d_1_func_d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_1.H.func.m")
ch5un_fwi2d_1_func_m = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_3a.H.func")
ch5un_fwi2d_3a_func = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_3a.H.func.d")
ch5un_fwi2d_3a_func_d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_3a.H.func.m")
ch5un_fwi2d_3a_func_m = data.reshape(axes.n,order='F').T

In [ ]:
# remove unwanted (intermediate) trial values for the extended FWI

select_list=[]
for i in range(ch5un_fwi2d_3a_func_d.shape[0]):
    if int(100000*(ch5un_fwi2d_3a_func_d[i]+ch5un_fwi2d_3a_func_m[i])) in (100000*ch5un_fwi2d_3a_func).astype(int):
        select_list.append(i)
ch5un_fwi2d_3a_func_d = ch5un_fwi2d_3a_func_d[select_list]
ch5un_fwi2d_3a_func_m = ch5un_fwi2d_3a_func_m[select_list]

select_list=[]
for i in range(ch5un_fwi2d_1_func_d.shape[0]):
    if int(100000*(ch5un_fwi2d_1_func_d[i]+ch5un_fwi2d_1_func_m[i])) in (100000*ch5un_fwi2d_1_func).astype(int):
        select_list.append(i)
ch5un_fwi2d_1_func_d = ch5un_fwi2d_1_func_d[select_list]
ch5un_fwi2d_1_func_m = ch5un_fwi2d_1_func_m[select_list]

In [ ]:
# collect the misfit functions to be plotted together

xfunc = [np.arange(0,ch5un_fwi2d_2a_func.shape[0],dtype=int),
         np.arange(0,ch5un_fwi2d_3a_func_d.shape[0],dtype=int),
         np.arange(0,ch5un_fwi2d_2c_func.shape[0],dtype=int), 
         np.arange(0,ch5un_fwi2d_1_func_d.shape[0],dtype=int)
         ]

yfunc = [ch5un_fwi2d_2a_func, 
         ch5un_fwi2d_3a_func_d,
         ch5un_fwi2d_2c_func,
         ch5un_fwi2d_1_func_d
         ]

In [ ]:
plotfunc([xfunc[0],xfunc[1],xfunc[2],xfunc[3]], [yfunc[0]/yfunc[0][0],yfunc[1]/yfunc[1][0],yfunc[2]/yfunc[2][0],yfunc[3]/yfunc[3][0]], ymin=0, ymax=0, yscale="linear", ylabel="Normalized data misfit", labels=["Conventional","Extended","Conventional + BS","Extended + BS"], legend=True, savefig="ch5un_data_misfit")

### Seismic data

In [ ]:
# read seismic data

axes, data = sep.read_file(datapath+"ch5_data0_group2.H")
ch5un_data0_group2 = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_data_group2.H.bp30100")
ch5un_data_group2 = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_data_group2.H.bp30150")
ch5un_data_group2_hf = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5st_data_group2.H.bp30150")
ch5st_data_group2_hf = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_model_2d.H.syn.bp30100")
ch5un_model_2d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_model_2d.H.syn_hf.bp30150")
ch5un_model_2d_hf = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_3a.H.syn.bp30100")
ch5un_fwi2d_3a = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5un_fwi2d_4a.H.syn_hf.bp30150")
ch5un_fwi2d_4a_hf = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5st_model_2d.H.syn_hf.bp30150")
ch5st_model_2d_hf = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5st_fwi2d_2_syn_hf.H.bp30150")
ch5st_fwi2d_2_syn_hf = data.reshape(axes.n,order='F').T

nt=axes.n[0]
dt=axes.d[0]

In [ ]:
temp = ch5un_data0_group2[9][150:801,:]
p=0.25
vmax=p*np.amax(temp)
vmin=-vmax
fig=plt.figure(figsize=(6, 4),dpi=300)
plt.imshow(np.transpose(temp),interpolation='sinc',aspect="auto",extent=[-650,0,0.35,0],cmap='Greys',vmin=vmin,vmax=vmax)   
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.savefig(figpath+'ch5un_data0.png',bbox_inches='tight',format='png')

In [ ]:
temp = ch5un_data0_group2[14][801:1451,:]
p=0.25
vmax=p*np.amax(temp)
vmin=-vmax
fig=plt.figure(figsize=(6, 4),dpi=300)
plt.imshow(np.transpose(temp),interpolation='sinc',aspect="auto",extent=[0,650,0.35,0],cmap='Greys',vmin=vmin,vmax=vmax)   
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.savefig(figpath+'ch5st_data0.png',bbox_inches='tight',format='png')

In [ ]:
# data-synthetic interleave for the starting model
# plotseis_interleave(ch5un_data_group2[[14,11,8,5]], ch5un_model_2d[[14,11,8,5]] , inter=50 , savefig="ch5un_model_2d_interleave")
plotseis_interleave(ch5un_data_group2_hf[[14,11,8,5]], ch5un_model_2d_hf[[14,11,8,5]] , inter=50 , savefig="ch5un_model_2d_interleave_hf")

In [ ]:
# data-synthetic interleave for the final pre-stimulation FWI model
# plotseis_interleave(ch5un_data_group2[[14,11,8,5]], ch5un_fwi2d_3a[[14,11,8,5]] , inter=50 , savefig="ch5un_fwi2d_3a_interleave")
plotseis_interleave(ch5un_data_group2_hf[[14,11,8,5]], ch5un_fwi2d_4a_hf[[14,11,8,5]] , inter=50 , savefig="ch5un_fwi2d_4a_interleave_hf")

In [ ]:
# data-synthetic interleave for the starting model
plotseis_interleave(ch5st_data_group2_hf[[14,10,7,4]], ch5st_model_2d_hf[[14,10,7,4]] , xmin=150, xmax=650, inter=50 , savefig="ch5st_model_2d_interleave_hf")

In [ ]:
# data-synthetic interleave for the final post-stimulation FWI model
plotseis_interleave(ch5st_data_group2_hf[[14,10,7,4]], ch5st_fwi2d_2_syn_hf[[14,10,7,4]] , xmin=150, xmax=650, inter=50 , savefig="ch5st_fwi2d_2_interleave_hf")

### Matching filters

In [ ]:
# read data, filters and spectra
sp=8

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.H.bp30150")
dobs = data.reshape(data_axes.n,order='F').T
dobs=dobs[sp][400:451,:101]

nt=dobs.shape[1]
dt=data_axes.d[0]

data_axes, data = sep.read_file(datapath+"ch5un_syn_group2.H.bp30150")
dsyn = data.reshape(data_axes.n,order='F').T
dsyn=dsyn[sp][400:451,:101]

data_axes, data = sep.read_file(datapath+"ch5un_matched_uncst_group2.H.bp30150")
matched_un = data.reshape(data_axes.n,order='F').T
matched_un=matched_un[sp][:,:101]

data_axes, data = sep.read_file(datapath+"ch5un_matched_group2.H.bp30150")
matched_ct = data.reshape(data_axes.n,order='F').T
matched_ct=matched_ct[sp][:,:101]

data_axes, data = sep.read_file(datapath+"ch5un_matched_uncst_group2.H.bp30150.filter")
filter_un = data.reshape(data_axes.n,order='F').T
filter_un = filter_un[sp]

ntf=data_axes.n[0]

data_axes, data = sep.read_file(datapath+"ch5un_matched_group2.H.bp30150.filter")
filter_ct = data.reshape(data_axes.n,order='F').T
filter_ct = filter_ct[sp]

data_axes, data = sep.read_file(datapath+"ch5un_sources_group2.H")
src = data.reshape(data_axes.n,order='F').T
src = src[0,sp][:101]

data_axes, data = sep.read_file(datapath+"ch5un_sources_group2_matched30150_uncst.H")
src_un = data.reshape(data_axes.n,order='F').T
src_un = src_un[0,sp][:101]

data_axes, data = sep.read_file(datapath+"ch5un_sources_group2_matched30150.H")
src_ct = data.reshape(data_axes.n,order='F').T
src_ct = src_ct[0,sp][:101]

data_axes, data = sep.read_file(datapath+"ch5un_data_group2.H.bp30150.fx")
dobs_fx = data.reshape(data_axes.n,order='F').T
dobs_fx=np.sum(dobs_fx[sp],axis=0)
dobs_fx = dobs_fx/np.amax(dobs_fx)

nf=data_axes.n[0]
df=data_axes.d[0]

data_axes, data = sep.read_file(datapath+"ch5un_syn_group2.H.bp30150.fx")
dsyn_fx = data.reshape(data_axes.n,order='F').T
dsyn_fx=np.sum(dsyn_fx[sp],axis=0)
dsyn_fx = dsyn_fx/np.amax(dsyn_fx)

data_axes, data = sep.read_file(datapath+"ch5un_matched_uncst_group2.H.bp30150.fx")
matched_un_fx = data.reshape(data_axes.n,order='F').T
matched_un_fx=np.sum(matched_un_fx[sp],axis=0)
matched_un_fx = matched_un_fx/np.amax(matched_un_fx)

data_axes, data = sep.read_file(datapath+"ch5un_matched_group2.H.bp30150.fx")
matched_ct_fx = data.reshape(data_axes.n,order='F').T
matched_ct_fx=np.sum(matched_ct_fx[sp],axis=0)
matched_ct_fx = matched_ct_fx/np.amax(matched_ct_fx)

data_axes, data = sep.read_file(datapath+"ch5un_matched_uncst_group2.H.bp30150.filter.fx")
filter_un_fx = data.reshape(data_axes.n,order='F').T
filter_un_fx = filter_un_fx[sp]
filter_un_fx = filter_un_fx/np.amax(filter_un_fx)

nff=data_axes.n[0]
dff=data_axes.d[0]
off=data_axes.o[0]

data_axes, data = sep.read_file(datapath+"ch5un_matched_group2.H.bp30150.filter.fx")
filter_ct_fx = data.reshape(data_axes.n,order='F').T
filter_ct_fx = filter_ct_fx[sp]
filter_ct_fx = filter_ct_fx/np.amax(filter_ct_fx)

In [ ]:
# Filters and their spectra

plt.figure(figsize=(6.66,2),dpi=300)
plt.subplot(1,2,1)
plt.plot(filter_un,color='r',label="Basic")
plt.plot(filter_ct,color='b',linewidth=1,label="Preconditioned")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.xlim([0,30])
plt.title("a)",loc='left')
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%d'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})

plt.subplot(1,2,2)
plt.plot(np.linspace(0,(nff-1)*dff,nff),filter_un_fx,color='r',label="Basic")
plt.plot(np.linspace(0,(nff-1)*dff,nff),filter_ct_fx,color='b',linewidth=1,label="Preconditioned")
plt.xlabel("Frequency (Hz)")
# plt.ylabel("Amplitude")
plt.xlim([0,250])
plt.ylim([0,1.1])
plt.title("b)",loc='left')
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})

plt.tight_layout()

plt.savefig(figpath+'matching_filters.png',bbox_inches='tight',format='png')

In [ ]:
# seismic data

p=0.99
vmax=p*np.amax(dobs)
vmin=-vmax

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(dobs),interpolation='sinc',aspect="auto",extent=[-250,-200,0.2,0],cmap='Greys',vmin=vmin,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a)',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(dsyn),interpolation='sinc',aspect="auto",extent=[-250,-200,0.2,0],cmap='Greys',vmin=vmin,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b)',loc='left')

ax = plt.subplot(2,2,3)
plt.imshow(np.transpose(matched_un),interpolation='sinc',aspect="auto",extent=[-250,-200,0.2,0],cmap='Greys',vmin=vmin,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c)',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(matched_ct),interpolation='sinc',aspect="auto",extent=[-250,-200,0.2,0],cmap='Greys',vmin=vmin,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd)',loc='left')

plt.tight_layout(pad=0., w_pad=0., h_pad=0.5)

plt.savefig(figpath+'matching_data.png',bbox_inches='tight',format='png')

In [ ]:
# seismic data average spectra

plt.figure(figsize=(3.33,2),dpi=300)
plt.plot(np.linspace(0,(nf-1)*df,nf),dobs_fx,color='k',linewidth=1,label="Target")
plt.plot(np.linspace(0,(nf-1)*df,nf),dsyn_fx,'k--',linewidth=0.5,label="Initial")
plt.plot(np.linspace(0,(nf-1)*df,nf),matched_un_fx,color='r',linewidth=0.5,label="Basic")
plt.plot(np.linspace(0,(nf-1)*df,nf),matched_ct_fx,color='b',linewidth=0.5,label="Preconditioned")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.xlim([0,250])
plt.ylim([0,1.1])
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend(loc='upper right',prop={'size': 6})


plt.savefig(figpath+'matching_spectra.png',bbox_inches='tight',format='png')

In [ ]:
# matched seismic traces
itr=0

plt.figure(figsize=(6.66,2),dpi=300)

plt.plot(np.linspace(0,(nt-1)*dt,nt),dobs[itr],'k',linewidth=1,label="Target")
plt.plot(np.linspace(0,(nt-1)*dt,nt),dsyn[itr],'k--',linewidth=0.5,label="Initial")
plt.plot(np.linspace(0,(nt-1)*dt,nt),matched_un[itr],color='r',linewidth=0.5,label="Basic")
plt.plot(np.linspace(0,(nt-1)*dt,nt),matched_ct[itr],color='b',linewidth=0.5,label="Preconditioned")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.xlim([0,0.2])
plt.legend(loc='upper left',prop={'size': 6})
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.annotate("P", xy=[0.08,-1.2],xytext=[0.06,-1],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SV", xy=[0.165,-0.6],xytext=[0.175,-1.2],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

plt.savefig(figpath+'matching_traces.png',bbox_inches='tight',format='png')

In [ ]:
# source time functions

plt.figure(figsize=(6.66,2),dpi=300)

plt.plot(np.linspace(0,(nt-1)*dt,nt),src,'k',linewidth=1,label="Original")
plt.plot(np.linspace(0,(nt-1)*dt,nt),src_un,color='r',linewidth=0.5,label="Basic filtering")
plt.plot(np.linspace(0,(nt-1)*dt,nt),src_ct,color='b',linewidth=0.5,label="Preconditioned filtering")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.xlim([0,0.08])
plt.legend(loc='upper right',prop={'size': 6})
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.savefig(figpath+'matching_time_functions.png',bbox_inches='tight',format='png')